# NLP Project

In [18]:
import pandas as pd
import numpy as np

from requests import get
from bs4 import BeautifulSoup
import os

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_json("data.json")

In [17]:
df.language.value_counts()

JavaScript     4
C              2
Java           2
Shell          1
Objective-C    1
Name: language, dtype: int64

In [89]:
df

,repo,language,readme_contents
0,gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...
1,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w..."
2,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...
3,harthur/brain,JavaScript,*This project has reached the end of its devel...
4,google/agera,Java,![Agera](https://raw.githubusercontent.com/goo...
5,michaeltyson/TPKeyboardAvoiding,Objective-C,TPKeyboardAvoiding\n==================\n\nA dr...
6,bendc/sprint,JavaScript,"# Sprint.js\n\nSprint is a high-performance, 5..."
7,rengwuxian/RxJavaSamples,Java,样例代码已正式切换到基于 RxJava 2\n================\n\n> 需...
8,fastos/fastsocket,C,README for FASTSOCKET\n======================...
9,petruisfan/node-supervisor,JavaScript,# node-supervisor\n\nA little supervisor scrip...


# Acquire

Thanks to Zach, we already have the main function to acquire the data. The script runs through a list of repo's, and creates a new json file with the name of the repo, the programing language used, and the readme_contents.

In [68]:
df.head()

,repo,language,readme_contents
0,gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...
1,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w..."
2,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...
3,harthur/brain,JavaScript,*This project has reached the end of its devel...
4,google/agera,Java,![Agera](https://raw.githubusercontent.com/goo...


Now, we need to build a webscrapping function that can help us get more repo names, so we can run the script, and get their information. Our goal is to get 300 README's.

In [27]:
url = 'https://github.com/search?q=stars%3A%3E0&s=stars&type=Repositories'
headers = {'User-Agent': 'Codeup Bayes Data Science'} # codeup.com doesn't like our default user-agent
response = get(url, headers=headers)

In [28]:
soup = BeautifulSoup(response.content, 'html.parser')

In [44]:
article = soup.find_all('a', class_='v-align-middle')

In [61]:
article[0]["href"].replace("/", '', 1)

'ohmyzsh/ohmyzsh'

In [90]:
repo_names = []

for page_number in range(1,100):
    url = f"https://github.com/search?p={page_number}&q=stars%3A%3E0&s=stars&type=Repositories"
    headers = {'User-Agent': 'Codeup Bayes Data Science'} # codeup.com doesn't like our default user-agent
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    repos = soup.find_all('a', class_='v-align-middle')
    
    for repo_number in range(len(repos)):
        repo = repos[repo_number]["href"].replace("/", '', 1)
        repo_names.append(repo)

In [91]:
print(f"Number of repo names: {len(repo_names)}")
repo_names

Number of repo names: 90


['freeCodeCamp/freeCodeCamp',
 '996icu/996.ICU',
 'vuejs/vue',
 'EbookFoundation/free-programming-books',
 'facebook/react',
 'tensorflow/tensorflow',
 'twbs/bootstrap',
 'sindresorhus/awesome',
 'getify/You-Dont-Know-JS',
 'jwasham/coding-interview-university',
 'ohmyzsh/ohmyzsh',
 'kamranahmedse/developer-roadmap',
 'github/gitignore',
 'CyC2018/CS-Notes',
 'donnemartin/system-design-primer',
 'microsoft/vscode',
 'airbnb/javascript',
 'flutter/flutter',
 'd3/d3',
 'torvalds/linux',
 'public-apis/public-apis',
 'Snailclimb/JavaGuide',
 'jlevy/the-art-of-command-line',
 'TheAlgorithms/Python',
 'axios/axios',
 'golang/go',
 'trekhleb/javascript-algorithms',
 'nodejs/node',
 'ytdl-org/youtube-dl',
 'animate-css/animate.css',
 'axios/axios',
 'golang/go',
 'trekhleb/javascript-algorithms',
 'nodejs/node',
 'justjavac/free-programming-books-zh_CN',
 'ytdl-org/youtube-dl',
 'animate-css/animate.css',
 'kubernetes/kubernetes',
 'tensorflow/models',
 'microsoft/terminal',
 'moby/moby',
 '30